# Network anomaly detection
* <a href="#tag">Tag</a>

Using network data build a multiclass classification model to predict anomolies

- Jan 2021
- Domino Valdano and Frank McQuillan


## Index

### Setup 

* <a href="#dependencies">Dependencies</a>
* <a href="#package_options">Package Options</a>
* <a href="#database_connection">Database Connection</a>
    
    
### Data Loading

* <a href="#external_table">External Table Definition</a>
* <a href="#download_data">Download Data and View Sample</a>


### Data Audit

* <a href="#summary_statistics">Summary Statistics</a>


### Data Exploration

* <a href="#de_categorical">Categorical Columns</a>
* <a href="#de_continuous">Continuous Columns</a>


### Feature Engineering

* <a href="#fe_continuous">Continuous Features</a>
* <a href="#fe_one_hot">One Hot Encode Categorical Features</a>
* <a href="#fe_combine">Combine Continuous & Categorical Features</a>
* <a href="#fe_cats_dep">Plot Categorical Features By Response</a>
* <a href="#fe_chi_sq">Chi-squared Testing</a>
* <a href="#fe_corr">Correlation Testing</a>
* <a href="#fe_scatter">Scatter Plots</a>


### Model Development

* <a href="#train_vali_split">Training & Validation Sample Split</a>


* **Random Forest (MADlib)**
    * <a href="#rf_train_model">Train model</a>
    * <a href="#rf_variable_importance">Variable Importance</a>
    * <a href="#rf_score_out_of_sample">Score Validation Data</a>
    * <a href="#rf_auc">Area Under ROC Curve</a>
    * <a href="#rf_roc">Receiver Operating Characteristic Graph (ROC Curve)</a>
    * <a href="#rf_confusion_matrix">Confusion Matrix</a>
    * <a href="#rf_model_storage">Model Storage</a>

### Model Scoring


* <a href="#model_scoring_Example">Model Scoring Example</a>

----


## Setup

<a id="dependencies"></a>

In [ ]:
%load_ext sql

In [ ]:
# Greenplum Database 5.x on GCP (PM demo machine) - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

In [ ]:
# dependencies
import psycopg2               # Python-PostgreSQL Database Adapter - https://pypi.python.org/pypi/psycopg2
import pandas as pd           # Python Data Analysis Library - https://pandas.pydata.org/
import seaborn as sns         # Statistical data visualization - https://seaborn.pydata.org/
import math                   # Mathematical functions - https://docs.python.org/2/library/math.html
import textwrap as tw         # Text wrapping and filling - https://docs.python.org/2/library/textwrap.html
import ipywidgets as widgets  # Jupyter Widgets - https://ipywidgets.readthedocs.io/en/latest/
import IPython.display as ipd # http://ipython.org/documentation.html

<a id="package_options"></a>

In [ ]:
# package options
# %matplotlib inline
%pylab inline

pylab.rcParams['figure.figsize'] = (12, 8)
    
pd.options.mode.chained_assignment = None 
pd.set_option('display.max_colwidth', -1)

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000

sns.set(style="darkgrid")

<a id="database_connection"></a>

In [ ]:
# init to default values
database_host = 'localhost'
database_databasename = 'madlib'
database_username = 'gpadmin'
database_password = ''
database_port = '8000'

# interpret string as markdown
def printmd(string):
    ipd.display(ipd.Markdown(string))
    
# forms
message = "### Connection Details \n ------"
printmd(message)
    
printmd("**Host:**")
inputHost = widgets.Text()
ipd.display(inputHost)

printmd("**Port:**")
inputPort = widgets.Text()
ipd.display(inputPort)

printmd("**Database Name:**")
inputDatabaseName = widgets.Text()
ipd.display(inputDatabaseName)

printmd("**Username:**")
inputUsername = widgets.Text()
ipd.display(inputUsername)

printmd("**Password:**")
inputPassword = widgets.Text()
ipd.display(inputPassword)

printmd("*Leave blank for default values*")


def db_connect():
    global conn, cur
    try:
        connString = "host='{}' dbname='{}' user='{}' password='{}' port={}".format(database_host,database_databasename,database_username,database_password,database_port)
        # print connString
        conn = psycopg2.connect(connString)
        cur = conn.cursor()
        conn.autocommit = True
        message = "<span style='color:green'>**Connection successful!**</span>"
        printmd(message)
    except:
        message = "<span style='color:red'>**ERROR: Unable to connect to the database**</span>"
        printmd(message)
    
def on_button_click(b):
    
    global database_host, database_databasename, database_username, database_password, database_port
    
    ipd.clear_output()
    
    message = "### Connection Details \n ------"
    printmd(message)
    
    if inputHost.value == "":
        message = "**Host:** {} (default)".format(database_host)
        printmd(message)
    else:
        database_host = inputHost.value
        message = "**Host:** {}".format(database_host)
        printmd(message)

    if inputPort.value == "":
        message = "**Port:** {} (default)".format(database_port)
        printmd(message)
    else:
        database_port = inputPort.value
        message = "**Port:** {}".format(database_port)
        printmd(message)
        
    if inputDatabaseName.value == "":
        message = "**Database name:** {} (default)".format(database_databasename)
        printmd(message)
    else:
        database_databasename = inputDatabaseName.value
        message = "**Database name:** {}".format(database_databasename)
        printmd(message)
        
    if inputUsername.value == "":
        message = "**Username:** {} (default)".format(database_username)
        printmd(message)
    else:
        database_username = inputUsername.value
        message = "**Username:** {}".format(database_username)
        printmd(message)
        
    if inputPassword.value == "":
        message = "**Password:** {} (default)".format(database_password)
        printmd(message)
    else:
        database_password = inputPassword.value
        message = "**Password:** ###########"
        printmd(message)
    
    printmd("------")
    db_connect()
        
button = widgets.Button(description="Connect")
ipd.display(button)
button.on_click(on_button_click)

In [ ]:
def bar_plot(data,title,x,xLabel,y,yLabel,color=None,xAxisRotation=90):

    # Bar plot
    pylab.rcParams['figure.figsize'] = (12, 8)
    seq_col_brew = sns.color_palette("Blues_r", 1)
    sns.color_palette(seq_col_brew)
    if color != None:
        plt = sns.barplot(x=x, y=y, data=data, color=color)
    else:
        plt = sns.barplot(x=x, y=y, data=data)
        
    # titles
    plt.set_title(title,fontsize=30)
    plt.set_xlabel(xLabel,fontsize=12)
    plt.set_ylabel(yLabel,fontsize=12)
    
    # rotate x axis labels
    for item in plt.get_xticklabels():
        item.set_rotation(xAxisRotation)

    # remove scientific notation
    plt.ticklabel_format(style='plain', axis='y')


In [ ]:
database_host = 'localhost'
database_databasename = 'madlib'
database_username = 'gpadmin'
database_password = ''
database_port = '8000'
db_connect()

In [ ]:
# helper function
def query_gpdb(query): 

    cur.execute(query)

    colnames = [desc[0] for desc in cur.description]
    return pd.DataFrame(cur.fetchall(), columns=colnames)

## Data Loading

Training data

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data;
CREATE TABLE training_data (
    a1 TEXT,
    a2 TEXT,
    a3 TEXT,
    a4 FLOAT,
    a5 FLOAT,
    a6 INTEGER,
    a7 INTEGER,
    a8 INTEGER,
    a9 INTEGER,
    a10 INTEGER,
    a11 INTEGER,
    a12 FLOAT,
    a13 FLOAT,
    a14 FLOAT,
    a15 INTEGER,
    a16 FLOAT,
    a17 FLOAT,
    a18 FLOAT,
    a19 FLOAT,
    a20 FLOAT,
    a21 FLOAT,
    a22 FLOAT,
    a23 FLOAT,
    a24 INTEGER,
    a25 FLOAT,
    a26 FLOAT,
    a27 FLOAT,
    a28 FLOAT,
    a29 FLOAT,
    a30 FLOAT,
    a31 FLOAT,
    a32 FLOAT,
    a33 FLOAT,
    a34 FLOAT,
    a35 FLOAT,
    a36 FLOAT,
    a37 FLOAT,
    a38 FLOAT,
    a39 FLOAT,
    a40 FLOAT,
    a41 FLOAT,
    a42 INTEGER,
    y TEXT
);

COPY training_data FROM '/home/gpadmin/network_data/training.csv' CSV DELIMITER ',';

In [ ]:
%sql SET search_path=public,madlib

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_id;
CREATE TABLE training_data_id AS
SELECT ROW_NUMBER() OVER()-1 AS id, * FROM training_data;

In [ ]:
%%sql
DROP TABLE training_data;
ALTER TABLE training_data_id RENAME TO training_data;

In [ ]:
%%sql
SELECT * FROM training_data ORDER BY id LIMIT 5;

In [ ]:
%%sql
SELECT COUNT(*) FROM training_data;

Eval data

In [ ]:
%%sql
DROP TABLE IF EXISTS eval_data;
CREATE TABLE eval_data (
    id INTEGER,
    a1 TEXT,
    a2 TEXT,
    a3 TEXT,
    a4 FLOAT,
    a5 FLOAT,
    a6 INTEGER,
    a7 INTEGER,
    a8 INTEGER,
    a9 INTEGER,
    a10 INTEGER,
    a11 INTEGER,
    a12 FLOAT,
    a13 FLOAT,
    a14 FLOAT,
    a15 INTEGER,
    a16 FLOAT,
    a17 FLOAT,
    a18 FLOAT,
    a19 FLOAT,
    a20 FLOAT,
    a21 FLOAT,
    a22 FLOAT,
    a23 FLOAT,
    a24 INTEGER,
    a25 FLOAT,
    a26 FLOAT,
    a27 FLOAT,
    a28 FLOAT,
    a29 FLOAT,
    a30 FLOAT,
    a31 FLOAT,
    a32 FLOAT,
    a33 FLOAT,
    a34 FLOAT,
    a35 FLOAT,
    a36 FLOAT,
    a37 FLOAT,
    a38 FLOAT,
    a39 FLOAT,
    a40 FLOAT,
    a41 FLOAT,
    a42 INTEGER
);

COPY eval_data FROM '/home/gpadmin/network_data/eval-rev2-1.csv' DELIMITER ',' CSV HEADER;

In [ ]:
%%sql
SELECT * FROM eval_data ORDER BY id LIMIT 10;

In [ ]:
%%sql
SELECT COUNT(*) FROM eval_data;

## Pandas

In [ ]:
import pandas as pd

In [ ]:
training_data = pd.read_csv("/Users/fmcquillan/Documents/Technical/vmw_ml_fun_jan2021/training.csv", header=None) 

In [ ]:
training_data[42][:5]

In [ ]:
y_train = list(training_data[42][:5])

In [ ]:
headers = %sql select * from training_data limit 1;
headers = headers.DataFrame()

In [ ]:
training_data.columns = headers.columns[1:]

In [ ]:
training_data["y"][:5]

In [ ]:
training_data.drop(labels="y", axis=1)

In [71]:
X_train = training_data

In [ ]:
list(headers.columns)

In [ ]:
from imblearn.over_sampling import SMOTENC

In [ ]:
def sampling_strategy(header):
    n = [10] * (len(header)-1) # Just take 10 samples of each
    strat = { header[i] : n[i] for i in range(len(header)-1) }
    return strat

In [ ]:
cat_col_index = "a1 a2 a3 a6 a7 a8 a9 a10 a11 a15 a24 a42"
cat_col_index = cat_col_index.split()

In [ ]:
strat = sampling_strategy (list(headers.columns)[1:])

In [72]:
sm = SMOTENC(categorical_features=cat_col_index, random_state=123, sampling_strategy=strat)
X_trainres, y_trainres = sm.fit_resample(X_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [4898431, 5]

<a id="external_table"></a>

## Data Audit

<a id="summary_statistics"></a>
Summary Statistics

https://madlib.apache.org/docs/latest/group__grp__summary.html

In [ ]:
%%sql
DROP TABLE IF EXISTS training_summary;
SELECT * FROM madlib.summary( 'training_data',   -- Source table
                              'training_summary'    -- Output table
                            );

In [ ]:
%%sql
SELECT * FROM training_summary;

In [ ]:
%%sql
DROP TABLE IF EXISTS eval_summary;
SELECT * FROM madlib.summary( 'eval_data',   -- Source table
                              'eval_summary'    -- Output table
                            );

In [ ]:
%%sql
SELECT * FROM eval_summary;

## Data Exploration

<a id="de_categorical"></a>
#### Categorical Columns

In [ ]:
catColumns = ['a1','a2','a3', 'a6','a7','a8', 'a9',
              'a10','a11', 'a15','a24','a42',
              'y']

def bar_plot(data,title,x,xLabel,y,yLabel,color=None,xAxisRotation=90):

    # Bar plot
    pylab.rcParams['figure.figsize'] = (12, 8)
    seq_col_brew = sns.color_palette("Blues_r", 1)
    sns.color_palette(seq_col_brew)
    if color != None:
        plt = sns.barplot(x=x, y=y, data=data, color=color)
    else:
        plt = sns.barplot(x=x, y=y, data=data)
        
    # titles
    plt.set_title(title,fontsize=30)
    plt.set_xlabel(xLabel,fontsize=12)
    plt.set_ylabel(yLabel,fontsize=12)
    
    # rotate x axis labels
    for item in plt.get_xticklabels():
        item.set_rotation(xAxisRotation)

    # remove scientific notation
    plt.ticklabel_format(style='plain', axis='y')


def get_cat_data_frame(col):
    query = """
        SELECT *
              ,round((record_count * 100.0) / sum(record_count) OVER(),2) AS perc_records
        FROM (
            SELECT {} AS col
                  ,count(*) AS record_count
            FROM public.training_data
            GROUP BY 1
        ) foo
        ORDER BY perc_records DESC
    """.format(col)
    cur.execute(query)

    colnames = [desc[0] for desc in cur.description]
    return pd.DataFrame(cur.fetchall(), columns=colnames)
    
def on_cat_selection(res):
    if res['type'] == 'change' and res['name'] == 'value':
        ipd.clear_output()
        printmd("-----\n **Select Column:**")
        ipd.display(catDropdown)
        df = get_cat_data_frame(res['new'])
        bar_plot(df,res['new'],"col",res['new'],"perc_records","% Records", None, 0)
    
catDropdown = widgets.Dropdown(
    options=catColumns,
    value=catColumns[0],
    description='Column:',
    disabled=False,
)

catDropdown.observe(on_cat_selection)
printmd("-----\n **Select Column:**")
ipd.display(catDropdown)
df = get_cat_data_frame(catColumns[0])
bar_plot(df,catColumns[0],"col",catColumns[0],"perc_records","% Records", None, 0)

* Low % of values in any one class can skew model results and/or create unstable model. In practice we may consider merging and/or excluding some groups. (e.g. a4: [i], a5: [gg], a7: [z,j,dd,n,o), a13: [s,p])

<a id="de_continuous"></a>
#### Continuous Columns

In [ ]:
contColumns = ['a4', 'a5','a12', 'a13', 'a14', 'a16', 
               'a17', 'a18', 'a19', 'a20', 'a21', 
               'a22', 'a23', 'a25', 'a26', 'a27', 
               'a28', 'a29', 'a30', 'a31', 'a32', 
               'a33', 'a34', 'a35', 'a36', 'a37', 
               'a38', 'a39', 'a40', 'a41']
sliderValue = 20
colName = contColumns[0]

def get_cont_data_frame(col, buckets):
    query = """
        WITH aggs AS (
            SELECT min({c}) AS min,
                   max({c}) AS max
              FROM training_data
        )
        SELECT width_bucket({c}, min, max, {b}-1) AS bucket,
               ('[' || min({c}) || ',' || max({c}) || ')')::text as range,
               count(*) as freq
        FROM training_data, aggs
        GROUP BY bucket
        ORDER BY bucket
    """.format(c=col, b=buckets)
    cur.execute(query)

    colnames = [desc[0] for desc in cur.description]
    return pd.DataFrame(cur.fetchall(), columns=colnames)
    
def graph_reset():
    ipd.clear_output()
    printmd("-----\n")
    ipd.display(widgets.HBox((contDropdown,bucketsSlider)))
    printmd("-----\n")
    df = get_cont_data_frame(colName,sliderValue)
    bar_plot(df,colName,"range",colName,"freq","Frequency", "#4378E2")    
    
def on_cont_selection(res):
    global colName
    if res['type'] == 'change' and res['name'] == 'value':
        colName = res['new']
        graph_reset()
        
def on_slider_selection(res):
    global sliderValue
    if res['new'] == {} and res['old']:
        sliderValue = res['old']['value']
        graph_reset()
    
# Look at log transforms
#colsAddLogs = contColumns + ["log({} + 1)".format(c) for c in contColumns]
colsAddLogs = contColumns

contDropdown = widgets.Dropdown(
    options=colsAddLogs,
    value=colsAddLogs[0],
    description='Column:',
    disabled=False,
)

bucketsSlider = widgets.IntSlider(
    value=sliderValue,
    min=5,
    max=50,
    step=1,
    description='# Buckets:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

contDropdown.observe(on_cont_selection)
bucketsSlider.observe(on_slider_selection)

graph_reset()



* a15 large number of outliers - consider correcting
* Consider variable transformation if test non-tree based algorithm
* Histogram values are being calculated in the database - minimal data movement back to client

### Clustering

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_encoded, training_data_encoded_dictionary;
SELECT madlib.encode_categorical_variables (
        'training_data',            -- Source table
        'training_data_encoded',      -- Output table
        'a1, a2, a3'                        -- Categorical columns
        );

In [ ]:
%%sql
select * from training_data_encoded limit 5;

Sample training data

In [ ]:
%%sql
drop table if exists training_data_work1;
create table training_data_work1 as
(select * from training_data_encoded where y='class19' 
                                            or y='class12'
                                            or y='class13');

In [ ]:
%%sql
drop table if exists training_data_work2, training_data_work2_summary;
select madlib.cols2vec('training_data_work1',
                       'training_data_work2',
                       '*',
                       'y',
                       'y'
                       );

In [ ]:
%%sql
ALTER TABLE training_data_work2 ADD column pid serial;

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_work3;

CREATE TABLE training_data_work3 AS
(SELECT * FROM madlib.kmeanspp(
    'training_data_work2',                  -- points table
    'feature_vector',                      -- column name in point table
    3,
    'madlib.dist_norm1',   -- distance function
    'madlib.avg',                  -- aggregate function
    20,                            -- max iterations
    0.001                         -- minimum fraction of centroids reassigned to continue iterating
));

In [ ]:
%%sql
select * from training_data_work3;

In [ ]:
%%sql
DROP TABLE IF EXISTS point_cluster_map;
CREATE TABLE point_cluster_map AS
SELECT data.*, (madlib.closest_column(centroids, feature_vector, 'madlib.squared_dist_norm2')).*
FROM training_data_work2 as data, training_data_work3;
ALTER TABLE point_cluster_map RENAME column_id to cluster_id; -- change column name
SELECT y, pid, cluster_id, distance FROM point_cluster_map ORDER BY cluster_id, distance desc;

In [ ]:
%%sql
SELECT * FROM madlib.simple_silhouette( 'training_data_work2',          -- Input points table
                                              'feature_vector',              -- Points column in input table
                                              (select centroids from training_data_work3),           -- Column in centroids table containing centroids
                                              'madlib.squared_dist_norm2'   -- Distance function
                                      );

In [ ]:
%%sql
DROP TABLE IF EXISTS km_points_silh;
SELECT * FROM madlib.simple_silhouette_points( 'training_data_work2',          -- Input points table
                                              'km_points_silh',      -- Output table
                                              'pid',                 -- Point ID column in input table
                                              'feature_vector',              -- Points column in input table
                                              'training_data_work3',           -- Centroids table
                                              'centroids',           -- Column in centroids table containing centroids
                                              'madlib.squared_dist_norm2'   -- Distance function
                                      );
SELECT * FROM km_points_silh ORDER BY centroid_id;

whole table

In [ ]:
%%sql
drop table if exists training_data_work10, training_data_work10_summary;
select madlib.cols2vec('training_data_encoded',
                       'training_data_work10',
                       '*',
                       'y',
                       'y'
                       );

In [ ]:
%%sql
DROP TABLE IF EXISTS point_cluster_map;
CREATE TABLE point_cluster_map AS
SELECT data.*, (madlib.closest_column(centroids, feature_vector, 'madlib.squared_dist_norm2')).*
FROM training_data_work10 as data, training_data_work3;
ALTER TABLE point_cluster_map RENAME column_id to cluster_id; -- change column name

In [ ]:
%%sql
select * from point_cluster_map limit 5;

In [ ]:
%%sql 
select count(*) from point_cluster_map where cluster_id = 0 and distance < 130415.0;

In [ ]:
%%sql
DROP TABLE IF EXISTS k_auto, k_auto_summary;

SELECT madlib.kmeanspp_auto(
    'training_data_work2',                  -- points table
    'k_auto',                      -- output table
    'feature_vector',                      -- column name in point table
    ARRAY[2,3,4],              -- k values to try
    'madlib.squared_dist_norm2',   -- distance function
    'madlib.avg',                  -- aggregate function
    20,                            -- max iterations
    0.001,                         -- minimum fraction of centroids reassigned to continue iterating
    1.0,                           -- centroid seed
    'silhouette'                         -- k selection algorithm (silhouette or elbow or both)
);

In [ ]:
%%sql
SELECT * FROM k_auto_summary;

In [ ]:
%%sql
SELECT * FROM k_auto ORDER BY k;

In [ ]:
# get range of k values tested
k_range = %sql SELECT k FROM k_auto ORDER BY k;

# outer loop on k
# plot clusters for each k value
for n_clusters in k_range:
    
    # create table mapping each point to its centroid
    kval = n_clusters[0]
    %sql DROP TABLE IF EXISTS k_plot1;
    %sql CREATE TABLE k_plot1 AS (SELECT data.*, (madlib.closest_column(centroids, feature_vector, 'madlib.squared_dist_norm2')).column_id as cluster_id FROM training_data_work2 as data, k_auto WHERE k=$kval);

    # get info from tables and reshape to np arrays    
    # number of points
    num_points_proxy= %sql SELECT COUNT(*) FROM k_plot1;
    num_points= num_points_proxy[0][0]
    
    # points
    points_proxy = %sql SELECT feature_vector FROM k_plot1 ORDER BY pid;
    points = np.array(points_proxy).reshape(num_points,123) 
    
    # cluster id
    cluster_id_proxy = %sql SELECT cluster_id FROM k_plot1 ORDER BY pid;
    cluster_id = np.array(cluster_id_proxy).reshape(num_points)
    
    # centroids
    centroids_proxy = %sql SELECT centroids FROM k_auto WHERE k=$kval;
    centers = np.array(centroids_proxy[0][0]).reshape(kval,123)
    
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = %sql SELECT silhouette FROM k_auto WHERE k=$kval;
    print("For n_clusters =", kval,
          "The average silhouette_score is :", silhouette_avg)
    
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)
      
    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    len_X = %sql select count(*) from training_data_work2;
    len_X = len_X[0][0]
    ax1.set_ylim([0, len_X + (kval + 1) * 10])
    
    y_lower = 10
    
    # inner loop on number of centroids 
    for i in range(kval):

        %sql DROP TABLE IF EXISTS points_distr1;
        %sql SELECT * FROM madlib.simple_silhouette_points( 'training_data_work2', 'points_distr1', 'pid', 'feature_vector', (SELECT centroids FROM k_auto WHERE k=$kval), 'madlib.squared_dist_norm2');
        ith_cluster_silhouette_values_proxy = %sql SELECT silh from points_distr1 WHERE centroid_id=$i ORDER BY silh;
        ith_cluster_silhouette_values = np.array(ith_cluster_silhouette_values_proxy).reshape(len(ith_cluster_silhouette_values_proxy))
        
        size_cluster_i_proxy = %sql SELECT COUNT(*) from points_distr1 WHERE centroid_id=$i;
        size_cluster_i = size_cluster_i_proxy[0][0]

        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / kval)
        ax1.fill_betweenx(np.arange(y_lower, y_upper), 
                  0, ith_cluster_silhouette_values, 
                  facecolor=color, edgecolor=color, alpha=0.7);
        
        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i));

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
           
    ax1.set_title("Silhouette plot for the various clusters.")
    ax1.set_xlabel("Silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    
    # 2nd Plot showing the actual clusters formed
    centroids = %sql SELECT centroid_id FROM points_distr1 ORDER BY pid;
    cluster_labels = np.array(centroids).reshape(len(centroids))
    
    colors = cm.nipy_spectral(cluster_labels.astype(float) / kval)
    #ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
    #            c=colors, edgecolor='k')

    # Labeling the clusters
    # Draw white circles at cluster centers
    #ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
    #            c="white", alpha=1, s=200, edgecolor='k')

    #for i, c in enumerate(centers):
    #    ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
    #                s=50, edgecolor='k')

    #ax2.set_title("Visualization of the clustered data.")
    #ax2.set_xlabel("Feature space for the 1st feature")
    #ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % kval),
                 fontsize=14, fontweight='bold')

plt.show();

In [ ]:
len_X = %sql select count(*) from training_data_work2;

In [ ]:
len_X[0][0]

### Nearest neighbors - v1

In [ ]:
%%sql
drop table if exists temp1;
create table temp1 as
select * from training_data_encoded where y='class19' or y='class12';
select * from temp1;

In [ ]:
%%sql
DROP TABLE IF EXISTS temp1_summary;
SELECT * FROM madlib.summary( 'temp1',   -- Source table
                              'temp1_summary',    -- Output table
                              NULL,      -- summarize all columns
                              'y'    -- grouping 
                            );
select group_by_value, lower(target_column)::VARCHAR, column_number, mean from temp1_summary  where group_by='y' order by group_by, group_by_value, column_number;

In [ ]:
%%sql
DROP TABLE IF EXISTS temp2;
create table temp2 as
select group_by_value, target_column, mean from temp1_summary 
  where (target_column='a4' or target_column='a5') and group_by_value<>'None'
  order by group_by_value, target_column;
select * from temp2 order by group_by_value, target_column;

In [ ]:
%%sql
DROP TABLE IF EXISTS temp3;
SELECT madlib.pivot('temp2', 'temp3', 'group_by_value', 'target_column', 'mean');
SELECT * FROM temp3;

### Nearest neighbors - v2

In [ ]:
%%sql
drop table if exists temp1;
  create table temp1 as 
  select * from training_data;

In [ ]:
%%sql
drop table if exists temp2;
  create table temp2 as  
  select LOWER(a1) as a1, LOWER(a2) as a2, LOWER(a3) as a3, 
    a4,
    a5,
    a6 ,
    a7 ,
    a8 ,
    a9 ,
    a10 ,
    a11 ,
    a12 ,
    a13 ,
    a14 ,
    a15 ,
    a16 ,
    a17 ,
    a18 ,
    a19 ,
    a20 ,
    a21 ,
    a22 ,
    a23 ,
    a24 ,
    a25 ,
    a26 ,
    a27 ,
    a28 ,
    a29 ,
    a30 ,
    a31 ,
    a32 ,
    a33 ,
    a34 ,
    a35 ,
    a36 ,
    a37 ,
    a38 ,
    a39 ,
    a40 ,
    a41 ,
    a42 ,
    y from temp1;

In [ ]:
%%sql
select * from temp2 limit 5;

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_encoded, training_data_encoded_dictionary;
SELECT madlib.encode_categorical_variables (
        'temp2',            -- Source table
        'training_data_encoded',      -- Output table
        'a1, a2, a3'                        -- Categorical columns
        );

In [ ]:
%%sql
select * from training_data_encoded limit 5;

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_encoded_summary;
SELECT * FROM madlib.summary( 'training_data_encoded',   -- Source table
                              'training_data_encoded_summary',    -- Output table
                              NULL,      -- summarize all columns
                              'y',    -- grouping 
                             NULL,
                             NULL,
                             NULL,
                             NULL,
                             NULL,
                             5
                            );

In [ ]:
%%sql
select group_by, group_by_value, target_column, column_number, mean, variance, min, max from training_data_encoded_summary where target_column='a5' order by group_by, group_by_value, column_number;

In [ ]:
%%sql
drop table if exists training_data_encoded_summary_all;
create table training_data_encoded_summary_all as
  select * from training_data_encoded_summary where group_by is NULL;
select * from training_data_encoded_summary_all order by column_number;

In [ ]:
%%sql
drop table if exists work1;
create table work1 as 
  select * from training_data_encoded limit 10;
select * from work1;

In [ ]:
%%sql
drop table if exists drop table if exists work2;
create table work2 as
DROP TABLE IF EXISTS training_data_encoded_summary;

In [ ]:
%%sql
drop table if exists temp3;
create table temp3 as
select * from training_data_encoded where y='class19';
select * from temp3;

In [ ]:
%%sql
DROP TABLE IF EXISTS temp4;
create table temp4 as
select * from temp3_summary where group_by = 'y';

In [ ]:
%%sql
DROP TABLE IF EXISTS temp5;
SELECT madlib.pivot('temp4', 'temp5', 'group_by_value', 'target_column', 'mean');
SELECT * FROM temp5;

In [ ]:
%%sql
drop table if exists temp6, temp6_summary;
select madlib.cols2vec('temp5',
                       'temp6',
                       '*',
                       'group_by_value',
                       'group_by_value'
                       );

In [ ]:
%%sql
alter table temp6
rename column group_by_value to y;

In [ ]:
%%sql
select * from temp6;

In [ ]:
%%sql
drop table if exists temp7, temp7_summary;
select madlib.cols2vec('training_data_encoded',
                       'temp7',
                       '*',
                       'y',
                       'y'
                       );

In [ ]:
%%sql
select * from temp7 limit 5;

In [ ]:
%%sql
create table temp8 as
  select temp6.y as from_class , temp7.y as to_class, madlib.squared_dist_norm2(temp6.feature_vector, temp7.feature_vector) from temp6, temp7;

In [ ]:
%%sql
select * from temp8 order by squared_dist_norm2 asc limit 100;

----
## Feature Engineering

<a id="fe_continuous"></a>
#### Continuous Features

In [ ]:
# continuos features (seperated out incase feature transformations are required)
query = """
    DROP TABLE IF EXISTS public.model_inputs_cont;
    CREATE TABLE public.model_inputs_cont AS
    SELECT _id
          ,a16 AS approval
          ,a2
          ,a3
          ,a8
          ,a11
          ,a14
          ,a15
    FROM public.credit_application_data
    DISTRIBUTED BY (_id);
    SELECT * FROM public.model_inputs_cont LIMIT 0;
"""
cur.execute(query)

contFeatureNames = [desc[0] for desc in cur.description]
contFeatureNames.remove('_id')
contFeatureNames.remove('approval')

query = """
    SELECT *
    FROM public.model_inputs_cont
"""
df = query_gpdb(query)

df.head()

<a id="fe_one_hot"></a>
#### One Hot Encode Categorical Features

https://madlib.apache.org/docs/latest/group__grp__encode__categorical.html

In [ ]:
# encode categorical features
query = """
    DROP TABLE IF EXISTS public.model_inputs_cat;
    SELECT madlib.encode_categorical_variables (
        'public.credit_application_data',
        'public.model_inputs_cat',
        'a1,a4,a5,a6,a7,a9,a10,a12,a13',
        NULL,
        '_id',
        NULL,
        'a1=b, a4=y, a5=p, a6=x, a7=z, a9=false, a10=false, a12=false, a13=s'
    );
"""
cur.execute(query)

query = """
    SELECT *
    FROM public.model_inputs_cat
"""
cur.execute(query)

colnames = [desc[0] for desc in cur.description]
df = pd.DataFrame(cur.fetchall(), columns=colnames)

colnames.remove('_id')
catFeatureNames = colnames
featureNames = contFeatureNames + catFeatureNames
print(featureNames)
df.head()

<a id="fe_combine"></a>
#### Combine Continuous & Categorical Features

In [ ]:
# combine feature tables
query = """
    DROP TABLE IF EXISTS public.model_inputs;
    CREATE TABLE public.model_inputs AS
    SELECT *
    FROM public.model_inputs_cat
    JOIN public.model_inputs_cont
    USING (_id);
"""
cur.execute(query)

<a id="fe_cats_dep"></a>
#### Plot Categorical Features By Response

In [ ]:
def bar_plot_groupby(data, title,x,xLabel,y,yLabel,groupby,color=None,axisRotation=90):

    # Bar plot
    pylab.rcParams['figure.figsize'] = (12, 8)
    seq_col_brew = sns.color_palette("Blues_r", 1)
    sns.color_palette(seq_col_brew)
    if color != None:
        plt = sns.barplot(x=x, y=y, data=data, color=color, hue=groupby)
    else:
        plt = sns.barplot(x=x, y=y, data=data, hue=groupby)
        
    # titles
    plt.set_title(title,fontsize=30)
    plt.set_xlabel(xLabel,fontsize=16)
    plt.set_ylabel(yLabel,fontsize=16)
    
    # rotate x axis labels
    for item in plt.get_xticklabels():
        item.set_rotation(axisRotation)

    # remove scientific notation
    plt.ticklabel_format(style='plain', axis='y')

def get_cat_gb_data_frame(col):
    query = """
        SELECT *
              ,round((record_count * 100.0) / sum(record_count) OVER(PARTITION BY col),2) AS perc_records
        FROM (
            SELECT {} AS col
                  ,approval
                  ,count(*) AS record_count
            FROM public.model_inputs
            GROUP BY 1,2
        ) foo
        ORDER BY 1,2
    """.format(col)
    cur.execute(query)

    colnames = [desc[0] for desc in cur.description]
    return pd.DataFrame(cur.fetchall(), columns=colnames)
    
def on_cat_gb_selection(res):
    if res['type'] == 'change' and res['name'] == 'value':
        ipd.clear_output()
        printmd("-----\n **Select Column:**")
        ipd.display(catGPDropdown)
        df = get_cat_gb_data_frame(res['new'])
        bar_plot_groupby(df,res['new'],"col",res['new'],"perc_records","% Class Records", "approval")
    
catGPDropdown = widgets.Dropdown(
    options=catFeatureNames,
    value=catFeatureNames[0],
    description='Column:',
    disabled=False,
)

catGPDropdown.observe(on_cat_gb_selection)
printmd("-----\n **Select Column:**")
ipd.display(catGPDropdown)
df = get_cat_gb_data_frame(catFeatureNames[0])
bar_plot_groupby(df,catFeatureNames[0],"col",catFeatureNames[0],"perc_records","% Class Records","approval")

* a9_true appears to be a strong variable due to seperate between classes

<a id="fe_chi_sq"></a>
#### Chi-squared testing

https://en.wikipedia.org/wiki/Chi-squared_test

https://madlib.apache.org/docs/latest/group__grp__stats__tests.html

In [ ]:
def chi2_gof_test(feature_name, response):
    query = """
        WITH freq AS (
            SELECT {feature_name}
                  ,{response}
                  ,count(*) AS observed
            FROM public.model_inputs
            GROUP BY 1,2
        )
        SELECT '{feature_name}' AS feature_name
              ,'{response}' AS response
              ,(madlib.chi2_gof_test(observed, expected, deg_freedom)).*
        FROM (
            SELECT observed
                  ,sum(observed) OVER (PARTITION BY {feature_name})::DOUBLE PRECISION
                       * sum(observed) OVER (PARTITION BY {response}) AS expected
            FROM freq
        ) l, (
            SELECT (count(distinct {feature_name}) - 1) * (count(distinct {response}) - 1) AS deg_freedom
            FROM freq
        ) r;
    """.format(feature_name=feature_name, response=response)
    cur.execute(query)

    colnames = [desc[0] for desc in cur.description]
    return pd.DataFrame(cur.fetchall(), columns=colnames)

def chi2_gof_test_multi(feature_list, response):
    res = chi2_gof_test(feature_list[0], response)
    for i in range(1,len(feature_list)):
        res = res.append(chi2_gof_test(feature_list[i], response))
        
    return res

chi2_results = chi2_gof_test_multi(catFeatureNames, 'approval')
chi2_results.sort_values('phi', inplace=True)
ipd.display(chi2_results)
bar_plot(chi2_results, "Chi-Squared Testing","feature_name","Feature Name","phi","Test Statistic", "#4378E2")

<a id="fe_corr"></a>
#### Correlation Testing

https://madlib.apache.org/docs/latest/group__grp__correlation.html

In [ ]:
# calc correlations
query = """
    DROP TABLE IF EXISTS public.feature_correlations, public.feature_correlations_summary;
    SELECT madlib.correlation( 
        'public.training_data',
        'public.feature_correlations',
        '{}'
    );
    SELECT * 
    FROM public.feature_correlations
    ORDER BY column_position;
""".format(",".join(contFeatureNames))
corr = query_gpdb(query)

corr.drop('column_position', 'columns', inplace=True)
corr.set_index('variable', True, False, True)
corr

In [ ]:
%%sql
DROP TABLE IF EXISTS public.feature_correlations, public.feature_correlations_summary;
SELECT madlib.correlation( 
        'public.training_data',
        'public.feature_correlations'
);

In [ ]:
corr = %sql SELECT * FROM public.feature_correlations ORDER BY column_position;
corr = corr.DataFrame()
corr.drop('column_position', 'columns', inplace=True)
corr.set_index('variable', True, False, True)
corr

In [ ]:
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
#f, ax = plt.subplots(figsize=(11, 9))
f, ax = plt.subplots(figsize=(13, 11))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=False);

<a id="fe_scatter"></a>
#### Scatter Plots

In [ ]:
def sample_scatter(title, x, xLabel, y, yLabel, sampleSize):
    
    pylab.rcParams['figure.figsize'] = (8, 8)

    # Grab sample
    query = """
        SELECT count(*) AS n
        FROM public.model_inputs;
    """.format(",".join(contFeatureNames))
    cur.execute(query)

    colnames = [desc[0] for desc in cur.description]
    n = pd.DataFrame(cur.fetchall(), columns=colnames)['n'][0]
    limit = math.floor(n * sampleSize)
    
    query = """
        SELECT {} AS col1
              ,{} AS col2
        FROM public.model_inputs
        LIMIT {};
    """.format(x, y, limit)
    cur.execute(query)

    colnames = [desc[0] for desc in cur.description]
    sample = pd.DataFrame(cur.fetchall(), columns=colnames)    
    
    # Generate scatterplot
    if x == y:
        sample
    plt = sns.regplot(x="col1", y="col2", data=sample)
    
    # titles
    plt.set_title("\n".join(tw.wrap(title,50)),fontsize=16)
    plt.set_xlabel(xLabel,fontsize=16)
    plt.set_ylabel(yLabel,fontsize=16)

    # add 1000s commas
    plt.get_yaxis().set_major_formatter(
        matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y), ',')))
    plt.get_xaxis().set_major_formatter(
        matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y), ',')))

In [ ]:
x = contFeatureNames[0]
y = contFeatureNames[1]

def reset():
    ipd.clear_output()
    printmd("-----\n **Select Features:**")
    ipd.display(scatterDropdown1, scatterDropdown2)

    sample_scatter("{} by {}".format(x, y), x, x, y, y, 1)   
    
def os1(res):
    global x
    if res['type'] == 'change' and res['name'] == 'value':
        contFeatureNames.append(x)
        x = res['new']
        contFeatureNames.remove(x)
        reset()

def os2(res):
    global y
    if res['type'] == 'change' and res['name'] == 'value':
        y = res['new']
        reset()
                   
scatterDropdown1 = widgets.Dropdown(
    options=contFeatureNames,
    value=x,
    description='x:',
    disabled=False,
)

scatterDropdown2 = widgets.Dropdown(
    options=contFeatureNames,
    value=y,
    description='y:',
    disabled=False,
)

scatterDropdown1.observe(os1)
scatterDropdown2.observe(os2)

reset()


<a id="tag"></a>

## Model Training

### (OLD NOTEBOOK) Training & Validation Sample Split
<a id="train_vali_split"></a>
https://madlib.apache.org/docs/latest/group__grp__train__test__split.html

In [ ]:
# split training and validation set
# we are careful not to include the same customer in both sets
query = """
    DROP TABLE IF EXISTS public.model
                        ,public.model_train
                        ,public.model_test;
                        
    SELECT madlib.train_test_split(
        'public.model_inputs',
        'public.model',
        0.7,
        NULL,
        NULL,
        '*',
        FALSE,
        TRUE
    )
"""
cur.execute(query)

In [ ]:
query = """
    SELECT *
    FROM public.model_train
    LIMIT 5
"""
df = query_gpdb(query)
df

### Random Forest (MADlib)

Run #1

In [ ]:
%sql SET search_path=network_anomaly_run1,madlib

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_sample1;
SELECT madlib.balance_sample(
                              'public.training_data',             -- Source table
                              'training_data_sample1',      -- Output table
                              'y',           -- Class column
                              'uniform',           -- Uniform sample
                               2300);                -- Desired output table size

In [ ]:
%sql ALTER TABLE training_data_sample1 DROP COLUMN __madlib_id__;

Run #2

In [ ]:
%sql SET search_path=network_anomaly_run2,madlib

In [ ]:
%%sql 
DROP TABLE IF EXISTS y;
CREATE TABLE y AS SELECT y, (COUNT(*)>=1000) AS is_top FROM public.training_data GROUP BY y;

In [ ]:
%%sql DROP TABLE IF EXISTS training_data_top;
CREATE TABLE training_data_top AS
    SELECT *
FROM public.training_data JOIN y USING(y) WHERE is_top=TRUE;
SELECT COUNT(*) FROM training_data_top

In [ ]:
%%sql DROP TABLE IF EXISTS training_data_bottom;
CREATE TABLE training_data_bottom AS
    SELECT *
FROM public.training_data JOIN y USING(y) WHERE is_top=FALSE;
SELECT COUNT(*) FROM training_data_bottom

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_top_sample2;
SELECT madlib.balance_sample(
                              'training_data_top',              -- Source table
                              'training_data_top_sample2',      -- Output table
                              'y',                 -- Class column
                              'uniform',           -- Uniform sample
                               9000);              -- Desired output table size

In [ ]:
%sql SELECT y, COUNT(*) FROM training_data_top_sample2 GROUP BY y;

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_bottom_sample2;
SELECT madlib.balance_sample(
                              'training_data_bottom',              -- Source table
                              'training_data_bottom_sample2',      -- Output table
                              'y',                 -- Class column
                              'uniform',           -- Uniform sample
                               1400);              -- Desired output table size

In [ ]:
%sql SELECT y, COUNT(*) FROM training_data_bottom_sample2 GROUP BY y;

In [ ]:
%%sql
    DROP TABLE IF EXISTS training_data_balanced2;
    CREATE TABLE training_data_balanced2 AS
    SELECT * FROM training_data_top_sample2 UNION SELECT * FROM training_data_bottom_sample2;

In [ ]:
%sql SELECT y,count(*) FROM training_data_balanced2 GROUP BY y ORDER BY 2;

In [ ]:
%sql ALTER TABLE training_data_balanced2 DROP COLUMN is_top;

In [ ]:
%%sql
DROP TABLE IF EXISTS balanced2_train, balanced2_test;
SELECT madlib.train_test_split(
                                'training_data_balanced2',    -- Source table
                                'balanced2',            -- Output table
                                0.8,       -- Sample proportion
                                NULL,       -- Sample proportion
                                NULL,       -- Strata definition
                                NULL,       -- Columns to output
                                FALSE,     -- Sample without replacement
                                TRUE);    -- Do not separate output tables
SELECT y,COUNT(*) FROM balanced2_train GROUP BY y;

<a id="#tag"></a>

In [ ]:
%sql SELECT y,COUNT(*) FROM balanced2_test GROUP BY y;

Run #3

In [ ]:
%sql SET search_path=network_anomaly_run3,madlib

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_sample3;
SELECT madlib.balance_sample(
                              'public.training_data',             -- Source table
                              'training_data_sample3',      -- Output table
                              'y',           -- Class column
                              'uniform',           -- Uniform sample
                               230000);                -- Desired output table size

In [ ]:
%sql ALTER TABLE training_data_sample3 DROP COLUMN __madlib_id__;

In [ ]:
%%sql
DROP TABLE IF EXISTS balanced3_train, balanced3_test;
SELECT madlib.train_test_split(
                                'training_data_sample3',    -- Source table
                                'balanced3',            -- Output table
                                0.8,       -- Sample proportion
                                NULL,       -- Sample proportion
                                NULL,       -- Strata definition
                                NULL,       -- Columns to output
                                FALSE,     -- Sample without replacement
                                TRUE);    -- Separate output tables
SELECT y,COUNT(*) FROM balanced3_train GROUP BY y;

In [ ]:
%%sql
SELECT y,COUNT(*) FROM balanced3_test GROUP BY y;

Run #4

In [ ]:
%%sql
DROP TABLE IF EXISTS training_data_sample4;
SELECT madlib.balance_sample(
                              'public.training_data',             -- Source table
                              'training_data_sample4',      -- Output table
                              'y',           -- Class column
                               $$
                               class19=10, class12=10, class13=10, 
                               class09=20, class03=20, class08=20, class16=20, class05=20, 
                               class22=100, class07=100, class02=100, class04=100,
                               class14=1000, 
                               class20=5000, class21=5000, class01=5000, class11=5000, 
                               class15=20000, class06=20000, class17=20000,
                               normal=100000, class10=100000, class18=100000
                               $$);

In [ ]:
%sql ALTER TABLE training_data_sample4 DROP COLUMN __madlib_id__;

In [ ]:
%%sql
DROP TABLE IF EXISTS balanced4_train, balanced4_test;
SELECT madlib.train_test_split(
                                'training_data_sample4',    -- Source table
                                'balanced4',            -- Output table
                                0.8,       -- Sample proportion
                                NULL,       -- Sample proportion
                                NULL,       -- Strata definition
                                NULL,       -- Columns to output
                                FALSE,     -- Sample without replacement
                                TRUE);    -- Separate output tables
SELECT y,COUNT(*) FROM balanced4_train GROUP BY y;

In [ ]:
%%sql
SELECT y,COUNT(*) FROM balanced4_test GROUP BY y;

### Train model
<a id="rf_train_model"></a>
https://madlib.apache.org/docs/latest/group__grp__random__forest.html

In [ ]:
%sql ALTER TABLE balanced2_train DROP COLUMN __madlib_id__;

In [ ]:
%sql ALTER TABLE balanced2_test DROP COLUMN __madlib_id__;

In [ ]:
%%sql
DROP TABLE IF EXISTS rf_model4, rf_model4_summary, rf_model4_group;
SELECT madlib.forest_train(
            'training_data_sample4',  -- source table
            'rf_model4',              -- output table
            'id',                     -- unique row id
            'y',                      -- dependent var
            '*',                      -- indep var
            null,                     -- cols to exclude
            null,                     -- grouping
            10::integer,              -- num trees
            5::integer,               -- num random features
            true::boolean,            -- importance
            5::integer,               -- num permutations
            10::integer,              -- max tree depth
            3::integer,               -- min split
            1::integer,               -- min bucket
            10::integer,               -- num splits
            NULL,                     -- null handling
            TRUE                      -- verbose
        );

In [ ]:
%%sql
    SELECT gid, sample_id
    FROM rf_model4;

<a id="rf_variable_importance"></a>
#### Variable Importance

In [ ]:
query = """SET search_path=public, madlib;
    SELECT unnest(string_to_array(independent_varnames,',')) AS feature_name
          ,unnest(impurity_var_importance) AS impurity_feature_importance
          ,unnest(oob_var_importance) AS oob_feature_importance
    FROM rf_model4_group l
        ,rf_model4_summary r
    ORDER BY 2 DESC
"""

df = query_gpdb(query)
ipd.display(df.head(10))
bar_plot(df,"Feature Importance","feature_name",'Feature Name',"impurity_feature_importance","Feature Importance", "#4378E2")

In [ ]:
bar_plot(df,"Feature Importance","feature_name",'Feature Name',"oob_feature_importance","Feature Importance", "#4378E2")

In [ ]:
%%sql
DROP TABLE IF EXISTS feature_importance;
SELECT get_var_importance('rf_model4', 'feature_importance');
SELECT * FROM feature_importance ORDER BY impurity_var_importance DESC;

<a id="rf_score_out_of_sample"></a>
#### Score Validation/Training Data

https://madlib.apache.org/docs/latest/group__grp__random__forest.html

In [ ]:
# Evaluate validation dataset
query = """
    DROP TABLE IF EXISTS rf_model4_scored;
    SELECT madlib.forest_predict('rf_model4',
                                 'balanced4_test',
                                 'rf_model4_scored',
                                 'response');
                
    DROP TABLE IF EXISTS rf_model4_scored_tmp;
    CREATE TABLE rf_model4_scored_tmp AS
    SELECT *
    FROM rf_model4_scored
    JOIN balanced4_test
    USING (id);
    DROP TABLE rf_model4_scored;
    ALTER TABLE rf_model4_scored_tmp RENAME TO rf_model4_scored;
    SELECT * FROM rf_model4_scored LIMIT 10;
    
"""
cur.execute(query)

In [ ]:
# Evaluate training dataset
query = """
    DROP TABLE IF EXISTS rf_model4_scored;
    SELECT madlib.forest_predict('rf_model4',
                                 'balanced4_train',
                                 'rf_model4_scored',
                                 'response');
                
    DROP TABLE IF EXISTS rf_model4_scored_tmp;
    CREATE TABLE rf_model4_scored_tmp AS
    SELECT *
    FROM rf_model4_scored
    JOIN balanced4_train
    USING (id);
    DROP TABLE rf_model4_scored;
    ALTER TABLE rf_model4_scored_tmp RENAME TO rf_model4_scored;
    SELECT * FROM rf_model4_scored LIMIT 10;
    
"""
cur.execute(query)

In [ ]:
# Evaluate on entire 4.9-million "training" dataset
query = """
    DROP TABLE IF EXISTS rf_model1_scored;
    SELECT madlib.forest_predict('rf_model1',
                                 'public.training_data',
                                 'rf_model1_scored',
                                 'response');
                
    DROP TABLE IF EXISTS rf_model1_scored_tmp;
    CREATE TABLE rf_model1_scored_tmp AS
    SELECT *
    FROM rf_model1_scored
    JOIN public.training_data
    USING (id);
    DROP TABLE rf_model1_scored;
    ALTER TABLE rf_model1_scored_tmp RENAME TO rf_model1_scored;
    SELECT * FROM rf_model1_scored LIMIT 10;
    
"""
cur.execute(query)

In [ ]:
# Evaluate on 311K "eval" dataset
query = """
    DROP TABLE IF EXISTS rf_model1_scored;
    SELECT madlib.forest_predict('rf_model1',
                                 'public.eval_data',
                                 'rf_model1_scored',
                                 'response');
                
    DROP TABLE IF EXISTS rf_model1_scored_tmp;
    CREATE TABLE rf_model1_scored_tmp AS
    SELECT *
    FROM rf_model1_scored
    JOIN public.eval_data
    USING (id);
    DROP TABLE rf_model1_scored;
    ALTER TABLE rf_model1_scored_tmp RENAME TO rf_model1_scored;
    SELECT * FROM rf_model1_scored LIMIT 10;
    
"""
cur.execute(query)

In [ ]:
%%sql
select * from rf_model2_scored order by id limit 10;

In [ ]:
%%sql 
SET search_path=network_anomaly_run2, madlib;

In [ ]:
%%sql
DROP TABLE IF EXISTS confusion;
SELECT madlib.confusion_matrix( 'rf_model4_scored', 'confusion', 'estimated_y', 'y');
SELECT * FROM confusion ORDER BY "class";

In [ ]:
%%sql DROP TABLE IF EXISTS accuracy_by_class;
CREATE TABLE accuracy_by_class AS
    SELECT y,correct, total,correct::FLOAT/total as accuracy
FROM (SELECT y, COUNT(*) AS correct FROM rf_model4_scored WHERE y=estimated_y GROUP BY y) c JOIN
     (SELECT y,count(*) AS total FROM rf_model4_scored GROUP BY y) t USING(y);
SELECT * FROM accuracy_by_class ORDER BY y;

In [ ]:
%sql SELECT AVG(accuracy) AS balanced_accuracy FROM accuracy_by_class;

Build histogram by class if don't have label

In [ ]:
%%sql
select estimated_y, count(*) from rf_model3_scored group by estimated_y order by estimated_y;

# Rest of OLD NOTEBOOK

<a id="rf_auc"></a>
#### Area Under ROC Curve

In [ ]:
# auc
query = """
    DROP TABLE IF EXISTS public.model_test_scored_auc;
    SELECT madlib.area_under_roc(
        'public.model_test_scored'
       ,'public.model_test_scored_auc'
       ,'estimated_prob_1'
       ,'approval'
    )
"""
cur.execute(query)

query = """
    SELECT * 
    FROM public.model_test_scored_auc;
"""
auc = query_gpdb(query)['area_under_roc'][0]

message = """-----\n **AUC =** {:0.5f}""".format(auc)
printmd(message)

<a id="rf_roc"></a>
#### Receiver Operating Characteristic Graph (ROC Curve)

In [ ]:
# roc
query = """
    DROP TABLE IF EXISTS public.model_test_scored_roc;
    SELECT madlib.binary_classifier( 
        'public.model_test_scored'
       ,'public.model_test_scored_roc'
       ,'estimated_prob_1'
       ,'approval'
    );
"""
cur.execute(query)

query = """
    SELECT threshold
          ,fpr
          ,tpr
    FROM public.model_test_scored_roc
    ORDER BY 1
"""
df = query_gpdb(query)

# roc curve
pylab.rcParams['figure.figsize'] = (8, 8)

plt.figure()
lw = 2
plt.plot(df['fpr'], df['tpr'], color='darkgreen', lw=lw, label='AUC {:0.2f}'.format(auc))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show();

<a id="rf_confusion_matrix"></a>
#### Confusion Matrix

In [ ]:
# confusion matrix (inclusive)
cutoff = 0.5

query = """
        SELECT approval AS obs
              ,CASE WHEN estimated_prob_1 >= {} THEN 1 ELSE 0 END AS pred
              ,count(*) AS num
        FROM public.model_test_scored
        GROUP BY 1,2
        ORDER BY 1,2
    """.format(cutoff)

query_gpdb(query)

<a id="rf_model_storage"></a>
#### Model Storage

In [ ]:
query = """
        DROP TABLE IF EXISTS public.my_models;
        CREATE TABLE public.my_models (
            gid integer[]
           ,sample_id integer[]
           ,tree madlib.bytea8[]
           ,created timestamp
           ,team text
           ,owner text
           ,description text
           ,model_type text
           ,model_params text
           ,current boolean
           ,model_id serial
        );
        
        INSERT INTO public.my_models (
            SELECT array_agg(gid) AS gid
                  ,array_agg(sample_id) AS sample_id
                  ,array_agg(tree) AS tree
                  ,now() AS created
                  ,'Pivotal Data Science Atlanta' AS team
                  ,'Jarrod Vawdrey' AS owner
                  ,'This is an example credit scoring model' AS description
                  ,'MADlib random forest' AS model_type
                  ,'{num_trees= ,num_random_features= ,importance= ,num_permutations= ,max_tree_depth= ,min_split= ,min_bucket= ,num_splits= }' AS model_params
                  ,True AS current
            FROM public.rf_model_output   
        );
        
        SELECT gid
              ,sample_id
              ,created
              ,team
              ,owner
              ,description
              ,model_type
              ,model_params
              ,current
        FROM public.my_models
    """

query_gpdb(query)

## Model Scoring

<a id="model_scoring_Example"></a>
#### Model Scoring Example

In [ ]:
<a id="rf_train_model"></a>
featureNames = ['a2', 'a3', 'a8', 'a11', 'a14', 'a15', 'a1_a', 'a4_l', 'a4_u', 'a5_g', 'a5_gg', 'a6_aa', 'a6_c', 'a6_cc', 'a6_d', 'a6_e', 'a6_ff', 'a6_i', 'a6_j', 'a6_k', 'a6_m', 'a6_q', 'a6_r', 'a6_w', 'a7_bb', 'a7_dd', 'a7_ff', 'a7_h', 'a7_j', 'a7_n', 'a7_o', 'a7_v', 'a9_true', 'a10_true', 'a12_true', 'a13_g', 'a13_p']
    

def add_continuous_slider(n, default):
    tstr = "target_column == '{}'".format(n)
    minValue = math.floor(data_summary.query(tstr)['min'])
    minValueOrZero = min(0,float(minValue))
    maxValue = math.ceil(data_summary.query(tstr)['max'])
    return widgets.FloatSlider(
        value=default,
        min=minValueOrZero,
        max=maxValue,
        step=0.1,
        description="",
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )

def add_drop_down(n, default):
    
    query = """
        SELECT {} AS col
        FROM public.credit_application_data
        GROUP BY 1
        ORDER BY 1
    """.format(n)
    values = query_gpdb(query)['col']

    return widgets.Dropdown(
        options=values,
        value=default,
        description='',
        disabled=False,
    )

def add_widgets():
    
    message = "### Loan Application \n ------"
    printmd(message)
    
    myWidgets = []

    myWidgets.append({'a1':add_drop_down('a1','a')})
    myWidgets.append({'a2':add_continuous_slider('a2',35.43)})
    myWidgets.append({'a3':add_continuous_slider('a3',12.0)})
    myWidgets.append({'a4':add_drop_down('a4','u')})
    myWidgets.append({'a5':add_drop_down('a5','g')})
    myWidgets.append({'a6':add_drop_down('a6','q')})
    myWidgets.append({'a7':add_drop_down('a7','h')})
    myWidgets.append({'a8':add_continuous_slider('a8',14.0)})
    myWidgets.append({'a9':add_drop_down('a9',True)})
    myWidgets.append({'a10':add_drop_down('a10',True)})
    myWidgets.append({'a11':add_continuous_slider('a11',8.0)})
    myWidgets.append({'a12':add_drop_down('a12',False)})
    myWidgets.append({'a13':add_drop_down('a13','g')})
    myWidgets.append({'a14':add_continuous_slider('a14',0.0)})
    myWidgets.append({'a15':add_continuous_slider('a15',6590.0)})
    
    for widget in myWidgets:
        n = widget.keys()[0]
        printmd("**{}:**".format(n))
        ipd.display(widget[n])

    message = "------"
    printmd(message)
    
    return myWidgets

    
def create_model_input(myWidgets):

    checks = {}
    conts = []
    f = []
    
    for i in range(0,len(featureNames)):
        f.append(0.0)
    
    for feature in featureNames:
        if "_" in feature:
            key = feature[0:feature.find("_")]
            val = feature[feature.find("_")+1:len(feature)]
            if key in checks:
                checks[key].append(val)
            else:
                checks[key] = [val]
        else:
            conts.append(feature)
            
    for widget in myWidgets:
        n = widget.keys()[0]
        val = widget[n].value

        # lower case boolean strings
        if isinstance(val,np.bool_):
            val = str(val).lower()

        if n in checks:
            checkFlag = False
            for c in checks[n]:
                if c == val:
                    checkFlag = True
                    pos = featureNames.index("{}_{}".format(n,val))
                    f[pos] = 1.0   
                    
            # make all associated values 0
            if checkFlag == False:
                for feature in featureNames:
                    if "_" in feature and feature[0:feature.find("_")+1] == n:
                        pos = featureNames.index(feature)
                        f[pos] = 0.0   
        elif n in conts:
            pos = featureNames.index(n)
            f[pos] = val

    return f
        
def rf_score(modelInputs):
    
    print(modelInputs)
    print(featureNames)
    
    ddlString = "_id integer"
    for f in featureNames:
        ddlString = ddlString + ",{} float".format(f)

    query = """
        DROP TABLE IF EXISTS public.prod_example_data, public.prod_example_score;
        CREATE TABLE public.prod_example_data ({});
        INSERT INTO public.prod_example_data VALUES ({});
        DROP TABLE IF EXISTS public.model_test_scored_tmp;
        SELECT madlib.forest_predict('public.rf_model_output',
                                     'public.prod_example_data',
                                     'public.prod_example_score',
                                     'prob');
        SELECT * FROM public.prod_example_score;
    """.format(ddlString, ",".join(str(x) for x in modelInputs))

    score = float(query_gpdb(query)['estimated_prob_1'])
    
    message = "High"
    if score <= 0.5:
        message = "Low"
    elif score <= 0.75:
        message = "Average"
    
    return (score, message)
    
def on_appbutton_click(b):
    
    ipd.clear_output()
    
    message = "### Loan Approval Results \n------\n"
    printmd(message)
    
    modelInput = create_model_input(myWidgets)
    
    s, m = rf_score(modelInput)
    
    message = "**Approval Score:** {}".format(s)
    printmd(message)
    
    message = "*Your chances of being approved are '{}'*".format(m)
    printmd(message)
    
    cleanModelInputs = ""
    for i in range(0,len(featureNames)):
        cleanModelInputs = cleanModelInputs + "{} = {}\n\n".format(featureNames[i],modelInput[i])

    message = "**Model Inputs:** \n\n{}".format(cleanModelInputs)
    printmd(message)
    
    printmd("\n------")
    
myWidgets = add_widgets()
appbutton = widgets.Button(description="Calculate Approval")
ipd.display(appbutton)
appbutton.on_click(on_appbutton_click)
